# [Solution] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys
import pprint

pp = pprint.PrettyPrinter(indent=4)

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example:
import os

from typing import List
from dotenv import load_dotenv
import chromadb
from tavily import TavilyClient
from lib.agents import Agent
from lib.llm import LLM
from lib.tooling import tool

In [3]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry.
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
@tool(
    name="retrieve_game",
    description="Retrieve game information from the vector database",
)
def retrieve_game(query: str) -> List[dict]:
    """
    Retrieve game information from the vector database based on the query.

    Args:
        query (str): A question about the game industry.

    Returns:
        List[dict]: A list of dictionaries containing game information.
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")

    results = collection.query(
        query_texts=[query],
        n_results=3,
        include=['documents']
    )

    # Extract relevant information from the results
    games = []
    for doc in results['documents'][0]:
        game_info = {
            "Platform": doc.split("Platform: [")[1].split("]")[0],
            "Publisher": doc.split("Publisher: ")[1].split("\n")[0].strip(),
            "Name": doc.split("Name: ")[1].split("Release Date:")[0].strip(),
            "YearOfRelease": doc.split("Release Date: (")[1].split(")")[0],
            "Description": doc.split("Description: ")[1].strip(),
            "Genre": doc.split("Genre: ")[1].split("\n")[0].strip(),
        }
        games.append(game_info)

    return games

retrieve_game("What is a game for Nintendo Switch?")  # Example usage

[{'Platform': 'Nintendo Switch',
  'Publisher': 'Nintendo',
  'Name': 'Mario Kart 8 Deluxe',
  'YearOfRelease': '2017',
  'Description': 'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.',
  'Genre': 'Racing'},
 {'Platform': 'Wii',
  'Publisher': 'Nintendo',
  'Name': 'Wii Sports',
  'YearOfRelease': '2006',
  'Description': "A collection of sports games that utilize the Wii's motion controls, bundled with the console to showcase its capabilities.",
  'Genre': 'Sports'},
 {'Platform': 'GameCube',
  'Publisher': 'Nintendo',
  'Name': 'Super Smash Bros. Melee',
  'YearOfRelease': '2001',
  'Description': 'A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.',
  'Genre': 'Fighting'}]

#### Evaluate Retrieval Tool

In [5]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents,
#    it will analyze the usability of the documents to respond to that question.
#    args:
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
@tool(
    name="evaluate_retrieval",
    description="Evaluate the usability of retrieved documents for answering a question",
)
def evaluate_retrieval(question: str, retrieved_docs: List[str]) -> dict:
    """
    Evaluate the usability of retrieved documents for answering a question.

    Args:
        question (str): The original question from the user.
        retrieved_docs (List[str]): The list of retrieved documents most similar to the user query.

    Returns:
        dict: A dictionary containing evaluation results with keys 'useful' and 'description'.
    """
    llm = LLM(model="gpt-4o-mini", temperature=0.3)

    prompt = (
        "Your task is to evaluate if the documents are enough to respond to the query. "
        "Give a detailed explanation, so it's possible to take an action to accept it or not.\n\n"
        f"Question: {question}\n\n"
        "Retrieved Documents:\n" + "\n".join(retrieved_docs) + "\n\n"
        "Evaluate if these documents are useful to answer the question."
    )

    response = llm.invoke(prompt)

    # Assuming response is structured as follows:
    # {
    #     "useful": True/False,
    #     "description": "Detailed explanation of the evaluation"
    # }

    return response

#### Game Web Search Tool

In [6]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.
@tool(
    name="game_web_search",
    description="Search the web for game industry information",
)
def game_web_search(question: str) -> List[dict]:
    """
    Search the web for game industry information based on the question.

    Args:
        question (str): A question about the game industry.

    Returns:
        List[dict]: A list of dictionaries containing search results.
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    results = tavily_client.search(
        query=question,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Extract relevant information from the results
    formatted_results = {
        "answer": results.get("answer", ""),
        "results": results.get("results", [])
    }

    return formatted_results

game_web_search("What is the a game for Nintendo Switch?")  # Example usage

{'answer': 'Mario Kart 8 Deluxe, Animal Crossing: New Horizons, and Super Smash Bros. Ultimate are popular Nintendo Switch games. The Switch also supports backward compatibility with older Nintendo games.',
 'results': [{'url': 'https://www.nintendo.com/sg/games/switch/index.html?srsltid=AfmBOord17FnCkLTVnCX7Y_ONotJKrU_80cUM0KOqyZ8z3VxVOhFJZsD',
   'title': 'Nintendo Switch Games',
   'content': 'Ring Fit Adventure · Mario Kart 8 Deluxe · Animal Crossing: New Horizons · The Legend of Zelda: Tears of the Kingdom.',
   'score': 0.6780931,
   'raw_content': None},
  {'url': 'https://en.wikipedia.org/wiki/List_of_Nintendo_Switch_games_(0%E2%80%939)',
   'title': 'List of Nintendo Switch games (0–9) - Wikipedia',
   'content': '|  v  t  e Nintendo Switch |\n| --- |\n| Hardware |  Nintendo Game Card  Joy-Con  Pro Controller  Amiibo  Nintendo Labo  Poké Ball Plus |\n| Services |  System software  Mii  Nintendo eShop  Switch Online  Nintendo Account  My Nintendo | [...] | Oculus |  Oculus Rift

### Agent

In [7]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions
# Plug all Tools you developed
instructions=("""
    You are an AI Research Agent for the video game industry. "
    Your tasks include: "
    1) Answering questions using internal knowledge first. "
    2) Evaluating the sufficiency of this answer using the 'evaluate_retrieval' tool. "
    3) If the internal knowledge is insufficient, use the 'game_web_search' tool as a fallback. "
    4) You may use the 'retrieve_game' tool to fetch relevant documents from the vector database to support your answer. "
    
    Provide the final answer based on the best available source."
    Format your final output as follows:
    
    User question:
    <user's question>
    
    Final answer:
    <your full answer here>"
    
    Sources:
    - If based on local data, write: Local database
    - If based on web search, provide full clickable URLs like: 
      [Wikipedia - GameName](https://...)

    Tools used:
    - If no tools used, skip this section
    - If tools are used write them as bulletpoints e.g. 
      - evaluate_retrieval
      - game_web_search
      
    Make sure the response is factual, clear, and well-formatted.
    The final report must include your answer with citations (if any), and a list of tools used.
    """
)


agent = Agent(
    model_name="gpt-4o-mini",
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    instructions=instructions
)

#### Generate session_id to handle multiple queries in a session

In [8]:
import uuid
session_id = uuid.uuid4()
print(f"session_id = {session_id}")

session_id = 0a263756-0fdc-4f1c-8f58-6695e26d16ee


#### Invoke Agent

In [9]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
result = agent.invoke("When was Pokémon Gold and Silver released?", session_id=session_id)
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
User question:
When was Pokémon Gold and Silver released?

Final answer:
Pokémon Gold and Silver were released in Japan on November 21, 1999. They were later released in North America on October 16, 2000, and in Europe on April 6, 2001.

Sources:
- Local database

Tools used:
- None


In [10]:
pp.pprint(result.get_final_state())

{   'current_tool_calls': None,
    'instructions': '\n'
                    '    You are an AI Research Agent for the video game '
                    'industry. "\n'
                    '    Your tasks include: "\n'
                    '    1) Answering questions using internal knowledge '
                    'first. "\n'
                    '    2) Evaluating the sufficiency of this answer using '
                    'the \'evaluate_retrieval\' tool. "\n'
                    '    3) If the internal knowledge is insufficient, use the '
                    '\'game_web_search\' tool as a fallback. "\n'
                    "    4) You may use the 'retrieve_game' tool to fetch "
                    'relevant documents from the vector database to support '
                    'your answer. "\n'
                    '\n'
                    '    Provide the final answer based on the best available '
                    'source."\n'
                    '    Format your final output as follow

In [11]:
# - Which one was the first 3D platformer Mario game?
result = agent.invoke("Which one was the first 3D platformer Mario game?", session_id=session_id)
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
User question:
Which one was the first 3D platformer Mario game?

Final answer:
The first 3D platformer Mario game is "Super Mario 64," which was released for the Nintendo 64 on June 23, 1996, in Japan. It was groundbreaking for its open-world design and 3D gameplay mechanics, setting a new standard for 3D platformers.

Sources:
- Local database

Tools used:
- None


In [12]:
pp.pprint(result.get_final_state())

{   'current_tool_calls': None,
    'instructions': '\n'
                    '    You are an AI Research Agent for the video game '
                    'industry. "\n'
                    '    Your tasks include: "\n'
                    '    1) Answering questions using internal knowledge '
                    'first. "\n'
                    '    2) Evaluating the sufficiency of this answer using '
                    'the \'evaluate_retrieval\' tool. "\n'
                    '    3) If the internal knowledge is insufficient, use the '
                    '\'game_web_search\' tool as a fallback. "\n'
                    "    4) You may use the 'retrieve_game' tool to fetch "
                    'relevant documents from the vector database to support '
                    'your answer. "\n'
                    '\n'
                    '    Provide the final answer based on the best available '
                    'source."\n'
                    '    Format your final output as follow

In [13]:
# - Was Mortal Kombat X realeased for Playstation 5?
result = agent.invoke("Was Mortal Kombat X realeased for Playstation 5?", session_id=session_id)
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
User question:
Was Mortal Kombat X released for Playstation 5?

Final answer:
Mortal Kombat X was not released specifically for the PlayStation 5. It was launched on April 14, 2015, for platforms including PlayStation 4, Xbox One, and PC, prior to the release of the PS5. However, it is playable on the PlayStation 5 through backward compatibility, allowing players to enjoy the game on the newer console.

Sources:
- Local database

Tools used:
- evaluate_retrieval


In [14]:
pp.pprint(result.get_final_state())

{   'current_tool_calls': None,
    'instructions': '\n'
                    '    You are an AI Research Agent for the video game '
                    'industry. "\n'
                    '    Your tasks include: "\n'
                    '    1) Answering questions using internal knowledge '
                    'first. "\n'
                    '    2) Evaluating the sufficiency of this answer using '
                    'the \'evaluate_retrieval\' tool. "\n'
                    '    3) If the internal knowledge is insufficient, use the '
                    '\'game_web_search\' tool as a fallback. "\n'
                    "    4) You may use the 'retrieve_game' tool to fetch "
                    'relevant documents from the vector database to support '
                    'your answer. "\n'
                    '\n'
                    '    Provide the final answer based on the best available '
                    'source."\n'
                    '    Format your final output as follow

#### Show that the agent has a conversation history or context memory

In [15]:
result = agent.invoke("Summarize the last requests and show me a combined report.", session_id=session_id)
print(result.get_final_state()["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
User question:
1. When was Pokémon Gold and Silver released?
2. Which one was the first 3D platformer Mario game?
3. Was Mortal Kombat X released for Playstation 5?

Final answer:
1. Pokémon Gold and Silver were released in Japan on November 21, 1999. They were later released in North America on October 16, 2000, and in Europe on April 6, 2001.

2. The first 3D platformer Mario game is "Super Mario 64," which was released for the Nintendo 64 on June 23, 1996. It was groundbreaking for its open-world design and 3D gameplay mechanics, setting a new standard for 3D platformers.

3. Mortal Kombat X was not released specifically for the PlayStation 5. It was launched on April 14, 2015, for platforms including PlayStation 4, Xbox One, and PC, prior to the release of the PS5. However, it is playable on the PlayStation 5 through

### (Optional) Advanced